In [ ]:
import pandas as pd
import os
import sklearn as sk
import torch

In [ ]:
fichiers = is.listdir('data_science/data/')
data = pd.read_csv(fichiers[0])
for file in fichiers[1:]:
    match = pd.read_csv(file)
    pd.concat([data,match])